In [1]:
#pragma cling add_include_path("../stlab/libraries")

In [2]:
#include <mutex>
#include <string>
#include <unordered_set>
#include <functional>
#include <condition_variable>
#include <deque>
#include <thread>
#include <iostream>
#include <type_traits>

#define STLAB_DISABLE_FUTURE_COROUTINES 1
#include <stlab/concurrency/future.hpp>
#include <stlab/concurrency/immediate_executor.hpp>
#include <stlab/concurrency/default_executor.hpp>
#include <stlab/concurrency/utility.hpp>

In [3]:
using namespace stlab;

In [4]:
using namespace std;

In [5]:
namespace bcc {
    
class task {
    struct concept;

    template <class F>
    struct model;

    std::unique_ptr<concept> _self;

public:
    task() = default;
    //...

    template <class F> // F model void()
    task(F f);         // Need to implement

    void operator()(); // Need to implement
};
    
} // namespace bcc

In [6]:
using namespace bcc;

In [7]:
struct bcc::task::concept {
    virtual ~concept() {}
    virtual void invoke() = 0;
};

In [8]:
template <class F>
struct bcc::task::model final : concept {
    F _f;
    model(F f) : _f(move(f)) {}
    void invoke() override { _f(); }
};

In [9]:
template <class F>
bcc::task::task(F f) : _self(make_unique<model<F>>(move(f))) { }

In [10]:
namespace bcc {

void task::operator()() { _self->invoke(); }

} // namespace bcc

In [11]:
namespace bcc {

class sequential_process {
    // using task = function<void()>;

    mutex _mutex;
    condition_variable _condition;
    deque<task> _queue;
    bool _done = false;

    void run_loop();

    thread _thread{[this] { run_loop(); }};

public:
    ~sequential_process();
    void async(task f);
};
    
sequential_process::~sequential_process() {
    {
        lock_guard<mutex> lock(_mutex);
        _done = true;
    }
    _condition.notify_one();
    _thread.join();
}

void sequential_process::run_loop() {
    while (true) {
        task work;
        {
            unique_lock<mutex> lock(_mutex);

            while (_queue.empty() && !_done) {
                _condition.wait(lock);
            }

            if (_queue.empty()) return;

            work = move(_queue.front());
            _queue.pop_front();
        }
        work();
    }
}
    
void sequential_process::async(task f) {
    {
        lock_guard<mutex> lock(_mutex);
        _queue.push_back(move(f));
    }
    _condition.notify_one();
}
    
} // namespace bcc
    
template <class F> // F models R()
inline auto async_packaged(sequential_process& process, F&& f) {
    using result_t = std::result_of_t<std::decay_t<F>()>;
    
    auto task_future = stlab::package<result_t()>(stlab::immediate_executor, std::forward<F>(f));
    
    process.async(move(task_future.first));
    
    return move(task_future.second);
}

using namespace bcc;

# Continuations

- Recap
    - Callbacks
        - Must be known in advance
        - Require functional form transformations
    - C++11 futures
        - Do not compose
        - Block on `get()`

- Continuations combine the best features<sup>*</sup> of these two approaches for returning a value from a task
    - Do not need to be known in advance
    - Do not require functional transformations
    - Compose
    - Do not block

- Continuations are part of futures in the [concurrency TS](http://en.cppreference.com/w/cpp/experimental/concurrency) in `<experimental/future>`
    - For this class I'm using [`stlab::future<>`](http://stlab.cc/libraries/concurrency/future/future/) which has additional capabilities and minor differences in syntax
    - I'm participating in a standard committee workshop on April 26th to discuss the future of futures in C++20

- A continuation is a callback attached to a `future`, using [`.then()`](http://stlab.cc/libraries/concurrency/future/future/then.html)
    - `.then()` returns a new future that can be used for more continuations

In [12]:
{
auto p = async(default_executor, []{ return 42; });
    
auto q = p.then([](int x){ cout << x << endl; });
    
auto r = q.then([]{ cout << "done!" << endl; });

blocking_get(r); // <-- DON'T DO THIS IN REAL CODE!!!
}

42


done!


- Recall our interned string implementation with futures

In [13]:
namespace bcc {

struct shared_pool {
    unordered_set<string> _pool;
    sequential_process _process;
    
    auto insert(string) -> stlab::future<const string*>;
};
    
auto shared_pool::insert(string a) -> stlab::future<const string*> {
    return async_packaged(_process, [this, _a = move(a)]() mutable {
        return &*_pool.insert(move(_a)).first;
    });
}
    
}

```cpp
class interned_string {
    // struct shared_pool

    static auto pool() -> shared_pool& {
        static shared_pool result;
        return result;
    }

    shared_future<const std::string*> _string;
public:
    interned_string(string a) : _string(pool().insert(move(a))) {}
    
    auto str() const {
        return *_string.get(); // <---- BLOCKING!!!
    }
};
```

In [14]:
namespace {

class interned_string {
    // struct shared_pool

    static auto pool() -> shared_pool& {
        static shared_pool result;
        return result;
    }

    stlab::future<const string*> _string; // or std::experimental::shared_future
public:
    interned_string(string a) : _string(pool().insert(move(a))) {}
    
    auto str() const -> stlab::future<reference_wrapper<const string>> {
        return _string.then([](const string* p) { return cref(*p); });
    }
};
    
} // namespace

In [15]:
{
interned_string s("Hello World!"s);

auto done = s.str().then([](const string& s){
    cout << s << '\n';
});
    
blocking_get(done);
}

Hello World!


- Pros for continuations
    - Do not need to be known in advance
    - Do not require functional transformations
        - Straight forward transformation from synchronous to asynchronous code
    - Compose
    - Do not block
- Cons
    - Require more synchronization than callbacks
    - Execution context is ambiguous
        - _Immediate execution_ may happen either in the calling context or resolving context

- Multiple continuation can be attached to a single `future` (or `shared_future`)
    - This _splits_ execution
    - i.e.
        - "blur the document"
            - "then save the document"
            - "then rotate the document"
- A _join_ is accomplished using `when_all()`

In [16]:
{
auto p = async(default_executor, []{ return 42; });
auto q = async(default_executor, []{ return 5; });

auto done = when_all(default_executor, [](int x, int y){ cout << x + y << endl; }, p, q);
    
blocking_get(done);
}

47


- Joins are non-blocking
    - Attach continuations to the arguments
    - Keep an atomic count of how many arguments are resolved
    - Execute continuation on last resolve
- Splits and Joins allow us to construct arbitrary dependency DAGs

- Continuations are a form of a sequential process